In [2]:
import pandas as pd
df_ultimate_test = pd.read_table("artificial_test.data", sep=" ", header=None)
df_ultimate_test = df_ultimate_test.drop(df_ultimate_test.columns[500], axis=1) # kolumna 500 do wyrzucenia
df_ultimate_test

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,483,454,513,495,523,469,453,477,506,479,...,455,480,543,259,413,520,485,498,523,510
1,485,508,493,487,478,472,504,476,479,475,...,486,480,535,534,514,452,484,495,548,477
2,483,521,507,475,493,486,421,475,496,483,...,491,476,498,495,508,528,486,465,508,503
3,474,504,576,480,553,483,524,478,483,483,...,521,475,470,463,509,525,479,467,552,517
4,495,474,523,479,495,488,485,476,497,478,...,510,471,522,343,509,520,475,493,506,491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,493,458,503,478,517,479,472,478,444,477,...,469,475,485,443,517,486,474,489,506,506
596,481,484,481,490,449,481,467,478,469,483,...,506,485,508,599,498,527,481,490,455,451
597,485,485,530,480,444,487,462,475,509,494,...,442,474,502,368,453,482,478,481,484,517
598,477,469,528,485,483,469,482,477,494,476,...,473,476,453,638,471,538,470,490,613,492


In [3]:
df_data_X = pd.read_table("artificial_train.data", sep=" ", header=None)
df_data_X = df_data_X.drop(df_data_X.columns[500], axis=1)
df_data_X

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,485,477,537,479,452,471,491,476,475,473,...,477,481,477,485,511,485,481,479,475,496
1,483,458,460,487,587,475,526,479,485,469,...,463,478,487,338,513,486,483,492,510,517
2,487,542,499,468,448,471,442,478,480,477,...,487,481,492,650,506,501,480,489,499,498
3,480,491,510,485,495,472,417,474,502,476,...,491,480,474,572,454,469,475,482,494,461
4,484,502,528,489,466,481,402,478,487,468,...,488,479,452,435,486,508,481,504,495,511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,490,505,503,474,463,461,519,476,518,467,...,467,479,449,588,499,506,475,463,507,501
1996,480,475,476,480,495,482,515,479,480,484,...,464,474,473,424,454,570,476,493,465,485
1997,480,517,631,470,485,474,535,476,493,466,...,501,483,479,687,488,488,483,500,523,481
1998,484,481,505,478,542,477,518,477,510,472,...,487,483,526,750,486,529,484,473,527,485


In [4]:
df_data_Y = pd.read_table("artificial_train.labels", sep=" ", header=None)
df_data_Y

,0
0,-1
1,-1
2,-1
3,1
4,1
...,...
1995,1
1996,-1
1997,-1
1998,1


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
X_train, X_test, y_train, y_test = train_test_split(df_data_X, df_data_Y, test_size=0.33, random_state=4)

In [8]:
from sklearn.base import BaseEstimator
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest

class FeatureSelectorSwitcher(BaseEstimator):

    def __init__(self, estimator = PCA(n_components=7, svd_solver='full')):
        self.estimator = estimator
        
    def fit(self, X, y=None, **kwargs):
            self.estimator.fit(X, y)
            return self
            
    def predict(self, X, y=None):
            return self.estimator.predict(X)
            
    def predict_proba(self, X):
            return self.estimator.predict_proba(X)
            
    def score(self, X, y):
            return self.estimator.score(X, y)


In [16]:
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define PCA
pca = PCA(n_components=7, svd_solver='full')

# Define feature selection
selection = SelectKBest()

# Define classifier
rf = RandomForestClassifier()

my_pipeline = Pipeline(steps = [("model", FeatureSelectorSwitcher()), ("algorithm", rf)])

grid_params = [
    {'model__estimator': [PCA(n_components=7, svd_solver='full')]},
    {'model__estimator': [SelectKBest(),
]}
]

# Define pipelines
# pipeline1 = Pipeline([("pca", pca), ("rf", rf)])
# pipeline2 = Pipeline([("select", selection), ("rf", rf)])
# pipeline3 = Pipeline([("select", selection), ("pca", pca), ("rf", rf)])

# Define parameter grid
# param_grid = [
#     {
#         'pca__n_components': [10, 20, 30],
#         'rf__n_estimators': [10, 50, 100],
#         'pipeline': [pipeline1]
#     },
#     {
#         'select__k': [10, 20],
#         'rf__n_estimators': [10, 50, 100],
#         'pipeline': [pipeline2]
#     },
#     {
#         'select__k': [10, 20],
#         'pca__n_components': [10, 20, 30],
#         'rf__n_estimators': [10, 50, 100],
#         'pipeline': [pipeline3]
#     },
# ]

# Define grid search
grid_search = GridSearchCV(Pipeline([('pipeline', my_pipeline)]), grid_params, cv=5, verbose=10, n_jobs=-1)
my_pipeline.get_params()

# Fit grid search
#grid_search.fit(X_train, y_train)

{'memory': None,
 'steps': [('model', FeatureSelectorSwitcher()),
  ('algorithm', RandomForestClassifier())],
 'verbose': False,
 'model': FeatureSelectorSwitcher(),
 'algorithm': RandomForestClassifier(),
 'model__estimator__copy': True,
 'model__estimator__iterated_power': 'auto',
 'model__estimator__n_components': 7,
 'model__estimator__n_oversamples': 10,
 'model__estimator__power_iteration_normalizer': 'auto',
 'model__estimator__random_state': None,
 'model__estimator__svd_solver': 'full',
 'model__estimator__tol': 0.0,
 'model__estimator__whiten': False,
 'model__estimator': PCA(n_components=7, svd_solver='full'),
 'algorithm__bootstrap': True,
 'algorithm__ccp_alpha': 0.0,
 'algorithm__class_weight': None,
 'algorithm__criterion': 'gini',
 'algorithm__max_depth': None,
 'algorithm__max_features': 'sqrt',
 'algorithm__max_leaf_nodes': None,
 'algorithm__max_samples': None,
 'algorithm__min_impurity_decrease': 0.0,
 'algorithm__min_samples_leaf': 1,
 'algorithm__min_samples_split